In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import pandas as pd

# Add the directory containing the library to sys.path
import os
import sys
library_path = os.path.abspath(r'C:\git\foraging-strategies\code')
if library_path not in sys.path:
    sys.path.append(library_path)
    
from tools_fixed import PatchForager

### **Define the forager model**

In [ ]:
def depletion_func(patch_id, t):
    return a[patch_id] * (b[patch_id] ** (-c[patch_id]*t)+d[patch_id])
    
def make_pr_plot(patch_list, best_time, max_stops=20):
    time_steps = np.arange(max_stops)
    
    # Plot the depletion rate over time
    plt.figure(figsize=(2, 2))
    ax = plt.subplot(111)

    # Generate a color map with unique colors for each patch
    color_map = plt.get_cmap('tab20')  # You can choose a different colormap if needed
    colors = color_map(np.linspace(0, 1, len(patch_list)))

    for patch_id in patch_list: 

        # Get the color for this patch
        color = colors[patch_id]
        
        # Compute the depletion rate for each time step
        p_R = [depletion_func(patch_id,t) for t in time_steps]
    
        plt.plot(time_steps+1, p_R,color=color)#label = str(p_R[best_time[patch_id]-1]), 
        plt.plot(best_time[patch_id], p_R[best_time[patch_id]-1],'o', color=color,
         label=f'Patch {patch_id}: {best_time[patch_id]} rewards')
    
    plt.xlabel('# Rewards')
    plt.ylabel('Probability of Reward')
    plt.legend(loc='center right', bbox_to_anchor=(1, 1.15))
    ax.spines[['right', 'top']].set_visible(False)
    # plt.savefig(f'figs/mvt_curves'+str(travel_time)+'.png', bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
dict_optimal = {}

# Initial probabilities of reward in each patch
travel_time = 3
reward_value = [5, 5]
a = [0.9, 0.6]
b = [2.76, 2.76]
c = [0.1278, 0.1278]
d = [0, 0]
indep_var = 'rewards' # Independent variables for the depletion function
# # Example usage
type_patches = len(c)
num_patches = 40

# #Three alternative ways to generate a list of patches
# patch_list = list(range(type_patches)) #equal distribution of patches
patch_list = [random.randint(0, type_patches-1) for _ in range(num_patches)] #randomly generated list of patched
# # patch_list = data # import data from session {To Be Implemented..}

forager = PatchForager(travel_time, reward_value, a, b, c, d, prob=True, depl_fxn = 'exp', indep_var=[indep_var,indep_var])
mvt_optimal = forager.calculate_optimal_stops(patch_list)

print(f"Optimal stops: {mvt_optimal['optimal_stops']}")
print(f"Max reward rate: {mvt_optimal['max_reward_rate']}")

dict_optimal['rate'] = list(mvt_optimal['optimal_stops'])

make_pr_plot(patch_list = [0,1], best_time=[mvt_optimal['optimal_stops'][0],mvt_optimal['optimal_stops'][1]], max_stops=20)

# Plot the grid
plt.figure(figsize=(5, 4))
plt.imshow(mvt_optimal['reward_rate_grid'], aspect='auto', cmap='magma', vmin=1, vmax=2.5)
plt.colorbar(label='Reward Rate')
plt.title('Reward Rate Grid')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.grid(False)  # Turn off the default grid lines
plt.show()

In [ ]:
### Grid search for best parameters for rate strategy
# Create a directory for the simulation results
df_sum = pd.DataFrame(columns=['x', 'y', 'reward_rate', 'strategy'])
os.makedirs('data', exist_ok=True)
data_sum = pd.DataFrame()

# Create an HDF5 file

x = 10
y = 10
for x in np.arange(1, 5, 0.25):
    for y in np.arange(1, 5, 0.25):
        for strategy_info in [{'strategy': 'rate', 'params': {'target_reward_rate': [x, y]}}]:
            # Run the simulation for each strategy
            data, _ = forager.run_simulation(strategy_info['strategy'], patch_list, **strategy_info['params'], )
            data['strategy'] = strategy_info['strategy']
            data_sum = pd.concat([data_sum, data])
            df_sum.loc[len(df_sum)] = [x, y, _, strategy_info['strategy']] 

# Save the summary data to a CSV file
data_sum.to_csv('data/simulation_data.csv', index=False)
print(df_sum.loc[df_sum.strategy == 'rate']['reward_rate'].mean())

#### **Choose specific values**

In [ ]:
### Search for specific valyes
# Create a directory for the simulation results
df_sum = pd.DataFrame(columns=['x', 'y', 'reward_rate', 'strategy'])
os.makedirs('data', exist_ok=True)
data_sum = pd.DataFrame()

# Create an HDF5 file

x = 10
y = 10
for sim in np.arange(30):
    print(f"Simulation {sim}")
    for strategy_info in [{'strategy': 'stops', 'params': {'target_stops': [13, 7]}},
                            {'strategy': 'rewards', 'params': {'target_rewards': [6, 3]}},
                            {'strategy': 'failures', 'params': {'max_failures': [6, 4]}},
                            {'strategy': 'consec_failures', 'params': {'consec_failures': [3, 2]}},
                            {'strategy': 'rate', 'params': {'target_reward_rate': [2, 2]}}]:
        
        # Run the simulation for each strategy
        data, _ = forager.run_simulation(strategy_info['strategy'], patch_list, **strategy_info['params'], )
        data['strategy'] = strategy_info['strategy']
        data['session_no'] = sim
        data_sum = pd.concat([data_sum, data])
        
        df_sum.loc[len(df_sum)] = [x, y, _, strategy_info['strategy']] 

# Save the summary data to a CSV file
data_sum.to_csv('data/simulation_data_separate_odors.csv', index=False)
print(df_sum.loc[df_sum.strategy == 'rate']['reward_rate'].mean())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

heatmap_data = df_sum.loc[df_sum.strategy == 'rate'].pivot(index = 'x', columns = 'y', values =  'reward_rate')

max_value = heatmap_data.max().max()
max_cell = heatmap_data.stack().idxmax()
# print(f"The cell with the largest value is at {max_cell} \n with a value of {max_value}")
sns.heatmap(heatmap_data, cmap='magma', ax=ax)
ax.set_title('Reward Rate for ' + 'rate' + '\n' + str(max_cell) + ' with a value of ' + str(np.around(max_value, 2)))
ax.set_xlabel('Patch 1 values')
ax.set_ylabel('Patch 2 values')

### **Grid search to try all parameters and find the most optimal one**

In [ ]:
### Grid search for best parameters
# Create a directory for the simulation results
df_sum = pd.DataFrame(columns=['x', 'y', 'reward_rate', 'strategy'])
os.makedirs('data', exist_ok=True)

# Create an HDF5 file
with h5py.File('data/data.h5', 'w') as hf:
    for x in range(0, 20):
        for y in range (0,20):
            for strategy_info in [{'strategy': 'stops', 'params': {'target_stops': [x, y]}},
                                   {'strategy': 'rewards', 'params': {'target_rewards': [x, y]}},
                                    {'strategy': 'failures', 'params': {'max_failures': [x, y]}},
                                    {'strategy': 'consec_failures', 'params': {'consec_failures': [x, y]}}]:
                                    # {'strategy': 'rate', 'params': {'target_reward_rate': [x, y]}}]:
                # Run the simulation for each strategy
                data, _ = forager.run_simulation(strategy_info['strategy'], patch_list, **strategy_info['params'], )
                
                df_sum.loc[len(df_sum)] = [x, y, _, strategy_info['strategy']] 

In [ ]:
dict_optimal

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8))
for strategy, ax in zip(['stops', 'rewards', 'failures', 'consec_failures'], axes.flatten()):

    # Pivot the DataFrame to create a matrix
    heatmap_data = df_sum.loc[df_sum.strategy == strategy].pivot(index = 'x', columns = 'y', values =  'reward_rate')

    max_value = heatmap_data.max().max()
    max_cell = heatmap_data.stack().idxmax()
    # print(f"The cell with the largest value is at {max_cell} \n with a value of {max_value}")
    sns.heatmap(heatmap_data, cmap='magma', ax=ax)
    ax.set_title('Reward Rate for ' + strategy + '\n' + str(max_cell) + ' with a value of ' + str(np.around(max_value, 2)))
    ax.set_xlabel('Patch 1 values')
    ax.set_ylabel('Patch 2 values')
    dict_optimal[strategy] = list(max_cell)
    

plt.tight_layout()
plt.show()

### **Simulation of different strategies using the optimal fit**

In [ ]:
df_sum = pd.DataFrame(columns=['simulation', 'strategy', 'reward_rate', 'optimal'])
os.makedirs('data', exist_ok=True)

# Define the strategies and their parameters
strategy_struct = {
    'rewards_opt': {'strategy': 'rewards', 'params': {'target_rewards': dict_optimal['rewards']}}, 
    'rate': {'strategy': 'rate', 'params': {'target_reward_rate': dict_optimal['rate']}},
    'stops_opt': {'strategy': 'stops', 'params': {'target_stops': dict_optimal['stops']}},
    'failures_opt': {'strategy': 'failures', 'params': {'max_failures': dict_optimal['failures']}},
    'consec_failures_opt': {'strategy': 'consec_failures', 'params': {'consec_failures': dict_optimal['consec_failures']}},
    'consec_failures_same': {'strategy': 'consec_failures', 'params': {'consec_failures': [4,4]}},
    'stops_same': {'strategy': 'stops', 'params': {'target_stops': [10,10]}},
    'stops_sample': {'strategy': 'stops', 'params': {'target_stops': [50,50]}},
}

# Create an HDF5 file
with h5py.File('data/data.h5', 'w') as hf:
    for i in range(500):
        # Create a group for this simulation
        sim_group = hf.create_group(f'simulation_{i}')
        
        # Run the simulation for each strategy
        for strategy_name, strategy_info in strategy_struct.items():
            data, _ = forager.run_simulation(strategy_info['strategy'], patch_list, **strategy_info['params'])
            df_sum.loc[len(df_sum)] = [i, strategy_name, _, list(strategy_info['params'].values())[0]] 
                       
            # Save results
            data = data.replace({None: np.nan})
            dataset = sim_group.create_dataset(strategy_name, data=data.to_numpy())
            # Save column names as attributes
            dataset.attrs['columns'] = data.columns.tolist()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10, 4))
sns.boxplot(x ='strategy', y='reward_rate', hue='strategy', palette='tab10', data=df_sum, ax=ax[0])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.scatterplot(x ='strategy', y='reward_rate', data=df_sum, hue='strategy', palette='tab10', ax=ax[1])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
sns.despine()

In [ ]:
dfs = []
with h5py.File('data/data.h5', 'r') as hf:
    for sim_key in hf.keys():  # e.g. 'simulation_0'
        sim_group = hf[sim_key]
        sim_num = int(sim_key.split('_')[-1])
        
        for strategy_key in sim_group.keys():
            dataset = sim_group[strategy_key]
            data_array = dataset[()]
            columns = dataset.attrs['columns']
            df = pd.DataFrame(data_array, columns=columns)
            df['simulation'] = sim_num
            df['strategy'] = strategy_key
            dfs.append(df)

# Combine all into one big DataFrame
all_data = pd.concat(dfs, ignore_index=True)
all_data.to_csv('data/simulation_data_df.csv', index=False)

In [ ]:
step = 0
transition  = 2
sequence = {
           1: {"step": step, "patch_types": "all", "end_condition_value": 20, "first_stop": 0.5},
           2: {"step": transition, "patch_types":"single", "end_condition_value": 5, "first_stop": 1},
           3: {"step": transition, "patch_types": "delayed", "end_condition_value": 5, "first_stop": 1},
           4: {"step": transition, "patch_types": "single", "end_condition_value": 3, "first_stop": 1},
           5: {"step": transition, "patch_types": "noreward", "end_condition_value": 5, "first_stop": 1},
           6: {"step": transition, "patch_types": "all", "end_condition_value": [], "first_stop": 0.5}
           }

In [ ]:
sequence[1]['end_condition_value']

In [ ]:
# Function to read data from HDF5 file
def read_h5_data(file_path, simulation_number, strategy):
    with h5py.File(file_path, 'r') as hf:
        dataset = hf[f'simulation_{simulation_number}/{strategy}']
        return pd.DataFrame(dataset[:], columns=dataset.attrs['columns'])

# Set up the plotting style
plt.style.use('ggplot')

# File path
h5_file_path = 'data/data.h5'

# Get a list of distinct colors from matplotlib colormap
colors = plt.get_cmap('tab10')  # You can change 'tab10' to other colormaps if needed

# Create the color_set dictionary
color_set = {strategy: colors(i) for i, strategy in enumerate(strategy_struct.keys())}

## Exploring results from generated data

In [ ]:
plt.figure(figsize=(6, 4))
for sim_num in range(20):  # Assuming 20 simulations

    for strategy in strategy_struct.keys():
        # Read data for this simulation and strategy
        data = read_h5_data(h5_file_path, sim_num, strategy)
        
        # Plot cumulative reward over time
        plt.plot(data['time'], data['reward'].cumsum(), label=strategy.replace('_', ' ').title(),alpha = .3,
                 lw = 1,color = color_set[strategy])

    plt.xlabel('Time')
    plt.ylabel('Cumulative Reward')
    plt.title(f'Cumulative Reward over Time \n for Different Strategies \n Travel Time: {travel_time}s')
    
    if sim_num==0:
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        
    plt.grid(True)
    plt.tight_layout()
    
    # Save the plot
# plt.savefig(f'figs/cumulative_reward_all.png', bbox_inches='tight', dpi=300)
# plt.close()  # Close the figure to free up memory

In [ ]:
# Number of simulations
num_simulations = 20

# Plot: Reward Rate over Time
plt.figure(figsize=(6, 4))
for sim_num in range(num_simulations):
    for strategy in strategy_struct.keys():
        # Read data for this simulation and strategy
        data = read_h5_data(h5_file_path, sim_num, strategy)
        
        # Calculate reward rate
        cumulative_reward = data['reward'].cumsum()
        reward_rate = cumulative_reward / data['time']
        
        # Plot reward rate over time
        plt.plot(data['time'], reward_rate, label=strategy.replace('_', ' ').title() if sim_num == 0 else "", 
                 alpha=0.3, color=color_set[strategy], linewidth = 1)
    if sim_num == 0:
        plt.legend()

plt.xlabel('Time')
plt.ylabel('Reward Rate')
plt.title('Reward Rate over Time for Different Strategies')

plt.grid(True)
plt.tight_layout()

# Save the plot
# plt.savefig('figs/reward_rate_all.png', bbox_inches='tight', dpi=300)

In [ ]:
# Number of simulations
num_simulations = 20

# Plot: Time Spent in Each Patch
fig, axs = plt.subplots(1,4, figsize=(12, 4), sharey=True)

for variable, ax_flat in zip(['time_in_patch', 'rewards_in_patch', 'failures_in_patch', 'consecutive_failures'], axs.flatten()):
    ax = ax_flat
    for sim_num in range(num_simulations):
        for strategy in strategy_struct.keys():
            
            # Read data for this simulation and strategy
            data = read_h5_data(h5_file_path, sim_num, strategy)
            
            # Calculate time spent in each patch
            # if variable == 'time_in_patch':
            #     patch_times = data[data['patch_id'] != -1].groupby('patch_id')[variable].max()
            # else:
            #     patch_times = data[data['patch_id'] != -1].groupby('patch_id')[variable].mean()
            
            df_results = data[data['patch_id'] != -1].groupby(['patch_id', 'patch_entry_time'])[variable].max().reset_index()
            patch_times = df_results.groupby(['patch_id'])[variable].mean()            
            # Plot time spent in each patch
            ax.plot(patch_times.index, patch_times.values, label=strategy if sim_num == 0 else "", 
                    marker='o', alpha=0.3, color=color_set[strategy])

            ax.set_xlabel('Patch ID')
            ax.set_xlim([-.9,2.9])
            ax.set_title(variable)
# plt.ylabel('Time Spent in Patch')
# plt.title('Time Spent in Each Patch \n for Different Strategies')
if sim_num == 0:
    plt.legend()
plt.grid(True)
plt.tight_layout()
plt.legend(strategy_struct.keys(), bbox_to_anchor=(2.05,1), loc='upper right')

# plt.savefig(f'figs/patch_stops.png', bbox_inches='tight', dpi=300)

In [ ]:
# Function to add jitter to the y-axis
def add_jitter(series, jitter_amount=0.0):
    return series + np.random.uniform(-jitter_amount, jitter_amount, size=series.shape)

colors = ['tab:orange', 'tab:green']
cmaps = ['Oranges','Greens']

In [ ]:
from scipy.stats import linregress
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde

for strategy in strategy_struct.keys():
    fig, ax = plt.subplots(3, 2, figsize=(10, 12), sharex=True)
    plt.suptitle(strategy)
    # Read data for this simulation and strategy
    data = read_h5_data(h5_file_path, sim_num, strategy)
    for index, criteria in enumerate(['cumulative_patch_reward', 'failures_in_patch', 'consecutive_failures']):
        ax1 = ax[index]
        summary = data.groupby(
            ['patch_entry_time', 'patch_id']
        ).agg({criteria: 'max', 'time_in_patch': 'count'}).reset_index()
        
        if criteria == 'cumulative_patch_reward':
            summary['rewards'] = summary['cumulative_patch_reward']/5
            criteria = 'rewards'
            
        # Add jitter to the y-axis
        summary['criteria_jitter'] = add_jitter(summary[criteria])
        summary['time_in_patch_delivered_jitter'] = add_jitter(summary['time_in_patch'])

        for i in range(2):
            # sns.scatterplot(data=summary.loc[summary.patch_id == i], x='time_in_patch_delivered_jitter', y='criteria_jitter', color=colors[i], 
            # legend=False, ax=ax1[i])
            
            y = summary.loc[summary.patch_id == i]['criteria_jitter']
            x = summary.loc[summary.patch_id == i]['time_in_patch_delivered_jitter']

            # # Calculate the point density
            # xy = np.vstack([x,y])
            # z = gaussian_kde(xy)(xy)
            
            # Perform PCA to reduce dimensionality
            pca = PCA(n_components=2)
            xy = np.vstack([x, y]).T
            xy_transformed = pca.fit_transform(xy)

            # Calculate the point density using the transformed data
            try:
                z = gaussian_kde(xy_transformed.T)(xy_transformed.T)

                ax1[i].scatter(x, y, c=z, s=100, cmap=cmaps[i])
            except ValueError:
                ax1[i].scatter(x, y, s=100, color=colors[i])
                
            try:
                slope, intercept, r_value, p_value, std_err = linregress(summary.loc[summary.patch_id == i]['time_in_patch_delivered_jitter'], summary.loc[summary.patch_id == i]['criteria_jitter'])
                # Annotate the plot with the slope and intercept
                ax1[i].text(0.05, 0.95, f'Slope: {slope:.2f}\nIntercept: {intercept:.2f}', transform=ax1[i].transAxes, 
                        fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', edgecolor='black', facecolor='white'))
                # Plot the regression line
                x_vals = np.array(ax1[i].get_xlim())
                y_vals = intercept + slope * x_vals
                ax1[i].plot(x_vals, y_vals, '-', color=colors[i], alpha=0.5)
                ax1[i].set_xlim(-1,30)
            except:
                pass
            ax1[i].set_ylim(-1,15)
            ax1[i].set_ylabel(criteria)
            ax1[i].set_xlabel('Time in Patch')

    fig.savefig(f'{strategy}.png', bbox_inches='tight', dpi=300)

### **Simulate and explore within session statistics for different strategies**

In [ ]:
class SessionData:
    def __init__(self, simulated_data):
        self.df = simulated_data
        self.variables = ['time_in_patch', 'prob_reward', 'cumulative_patch_reward', 'failures_in_patch', 'consecutive_failures']
    
    def process_last_timesteps(self):
        # Group by patch visit and get the last row of each visit
        grouped = self.df[self.df['patch_id'] != -1].groupby((self.df['patch_id'] != self.df['patch_id'].shift()).cumsum())
        last_timesteps = grouped.last().reset_index(drop=True)
        return last_timesteps
    
    def plot_variables_by_patch(self):
        last_timesteps = self.process_last_timesteps()
        
        # Create a color map for patches
        unique_patches = last_timesteps['patch_id'].unique()
        color_map = plt.get_cmap('tab20')
        color_dict = {patch: color_map(i/len(unique_patches)) for i, patch in enumerate(unique_patches)}
        
        fig, axes = plt.subplots(2, 3, figsize=(14, 8), sharex=True)
        fig.suptitle('Variables at Patch Exit, Colored by Patch', fontsize=16)
        
        for var, ax1 in zip(self.variables, axes.flatten()):
            ax = ax1
            for patch in unique_patches:
                patch_data = last_timesteps[last_timesteps['patch_id'] == patch]
                ax.scatter(patch_data.index, patch_data[var], 
                           c=[color_dict[patch]], label=f'Patch {patch}', s=10)
            
            ax.set_ylabel(var.replace('_', ' ').title())
            ax.grid(True, linestyle='--', alpha=0.3)
            ax.legend()
        
        ax.set_xlabel('Patch Visit Number')
        
        plt.tight_layout()
        plt.show()

    def plot_overall_reward_rate(self):
        # Calculate cumulative sum of rewards
        cumulative_rewards = self.df['reward'].cumsum()
        
        # Use the 'time' column for total time
        total_time = self.df['time']
        
        # Calculate reward rate
        reward_rate = cumulative_rewards / total_time
        
        plt.figure(figsize=(3, 2))
        plt.plot(total_time, reward_rate)
        plt.xlabel('Time')
        plt.ylabel('Overall Reward Rate')
        # plt.title('Overall Reward Rate Throughout the Session')
        plt.grid(True, linestyle='--', alpha=0.7)
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Run the simulation for each strategy
results = {}
for strategy_name, strategy_info in strategy_struct.items():
    simulated_data, _ = forager.run_simulation(strategy_info['strategy'], patch_list, **strategy_info['params'])
    results[strategy_name] = data

    print(strategy_name)
    session = SessionData(simulated_data)
    session.plot_variables_by_patch()
    # session.plot_overall_reward_rate()